# Process Crow et al. data
This notebook does the following:

1. Select template experiment. This template experiment will be used in the next [notebook](2_identify_generic_genes_pathways.ipynb) to simulate experiments with the same experimental design but testing a different biological process.

2. Crow et al. data was downloaded using `explore_RNAseq_only_generic_genes/download_Crow_data.R` script that downloads expression data from https://github.com/PavlidisLab/gemmaAPI.R

Note: For the analysis exploring the RNA-seq only common DEGs we used the union of genes per experiment, which resulted in some samples having NaNs for some samples. For this analysis we are taking the intersection so that we can remove all NaNs to train.

3. Train VAE on processed data.

In [1]:
%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2
import os
import pandas as pd
import pickle
from ponyo import utils, train_vae_modules
from generic_expression_patterns_modules import process

Using TensorFlow backend.
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
# Set seeds to get reproducible VAE trained models
process.set_all_seeds()

### Set parameters for data processing

Most parameters are read from `config_filename`. We manually selected bioproject [GSE11352](https://www.ncbi.nlm.nih.gov/gds/?term=GSE11352[Accession]) as the template experiment, which contains breast cell lines treated with estradiol at 12H, 24H and 48H.

In [3]:
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))

# Read in config variables
config_filename = os.path.abspath(
    os.path.join(base_dir, "configs", "config_human_Crow.tsv")
)

params = utils.read_config(config_filename)

local_dir = params["local_dir"]
dataset_name = params["dataset_name"]

# File that contains gene ranks identified by Crow et. al.
DE_prior_filename = params["reference_gene_filename"]

# Template experiment ID
project_id = params["project_id"]

# Column name containing sample ids
metadata_colname = params["metadata_colname"]

# Output file: pickled list of shared genes(generated during gene ID mapping)
shared_genes_filename = params["shared_genes_filename"]

# Output files of template experiment data
mapped_template_filename = params["mapped_template_filename"]
processed_template_filename = params["processed_template_filename"]

# Output files of Rani's compendium data
raw_compendium_filename = params["raw_compendium_filename"]
mapped_compendium_filename = params["mapped_compendium_filename"]
normalized_compendium_filename = params["normalized_compendium_filename"]

# Output file: pickled scaler (generated during compendium normalization)
scaler_filename = params["scaler_filename"]

### Load compendium data

In [4]:
raw_compendium = pd.read_csv(raw_compendium_filename, sep="\t", header=0, index_col=0)
print(raw_compendium.shape)
raw_compendium.head()

(32476, 14487)


,HSPA6|HSPA7,UBA7,ESRRA,CYP2A7|CYP2A6,ADAM32,CORO6,CCDC65,LEAP2,CTCFL,C4orf33,...,TRIM15,NBL1|MICOS10-NBL1,GRK2,ATN1,FBXO41,KCTD13,TFEB,DND1,CASKIN2,GIMAP1-GIMAP5|GIMAP5
L_IB38___BioAssayId=72907Name=050705MJA_U133_2.0_IB08.CEL,8.96564,83.32761,48.94783,6.05333,17.74886,177.35550,22.08059,24.29061,NaN,57.44873,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
L_IB40___BioAssayId=72909Name=050705MJA_U133_2.0_IB10.CEL,769.30450,61.90878,119.33116,17.42643,11.58205,117.40292,46.72446,50.81827,4.21085,29.51632,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
L_IB41___BioAssayId=72910Name=050705MJA_U133_2.0_IB11.CEL,23.69639,93.37774,80.67838,27.73698,18.56551,175.07208,21.01308,56.90577,4.52370,70.19458,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
L_IB33___BioAssayId=72902Name=050705MJA_U133_2.0_IB03.CEL,10.37628,56.58948,64.83450,14.86978,8.71849,41.07041,24.34623,62.00211,12.89886,38.07436,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
L_IB36___BioAssayId=72905Name=050705MJA_U133_2.0_IB06.CEL,48.51214,41.29277,76.53021,15.66673,5.35315,27.70762,4.19627,13.33500,0.54027,14.19567,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Process compendium data

1. Drop probe column
2. Transpose
3. Get only shared genes from Crow et. al.
4. Normalize

In [5]:
# Matrix is sample x gene
# Note, there are NaNs in the matrix. I'm not exactly sure what is causing this since the data
# downloaded was based on what Crow et al used in their analysis and therefore should have been
# filtered by platform.
# All genes have at least 1 NaN so dropping all genes with NaN removes all the data
# Instead we will move genes if they are NaN in _most_ samples (>90%)
x = raw_compendium.isna().sum()

processed_compendium = raw_compendium[x.index[x < 3000]]

# Get only gene expression data for genes in Crow et. al.
our_gene_ids_hgnc = list(processed_compendium.columns)

published_generic_genes = process.get_published_generic_genes(DE_prior_filename)
shared_genes_hgnc = list(set(our_gene_ids_hgnc).intersection(published_generic_genes))

# In Python, the order of elements in a list that is converted from a set
# is non-deterministic, so it is sorted here to have reproducible result.
shared_genes_hgnc.sort()

# Pickle `shared_genes_hgnc` and save as `shared_genes_filename`
if not os.path.exists(shared_genes_filename):
    with open(shared_genes_filename, "wb") as pkl_fh:
        pickle.dump(shared_genes_hgnc, pkl_fh)

mapped_compendium = processed_compendium[shared_genes_hgnc]
print(mapped_compendium.shape)
mapped_compendium.head()

(32476, 9240)


,A1BG,A1BG-AS1,A2M,A2M-AS1,A2ML1,A4GALT,A4GNT,AA06,AAAS,AACS,...,ZSWIM2,ZSWIM3,ZSWIM5,ZSWIM6,ZSWIM8-AS1,ZW10,ZWINT,ZXDA,ZXDB,ZYG11A
L_IB38___BioAssayId=72907Name=050705MJA_U133_2.0_IB08.CEL,11.63699,NaN,676.43811,18.94505,8.59358,NaN,19.44703,24.31123,50.73169,141.09604,...,NaN,4.83527,67.54955,470.20654,NaN,71.50054,59.74241,37.28952,37.95333,NaN
L_IB40___BioAssayId=72909Name=050705MJA_U133_2.0_IB10.CEL,20.67487,NaN,591.69562,23.72454,9.03128,NaN,47.68120,14.11284,49.37021,62.37345,...,NaN,16.71398,19.14305,325.37585,NaN,68.06551,27.87321,32.73832,38.87195,NaN
L_IB41___BioAssayId=72910Name=050705MJA_U133_2.0_IB11.CEL,40.67805,NaN,1069.66284,50.84652,8.31981,NaN,12.64032,14.81776,109.17731,237.93060,...,NaN,24.07349,66.29779,855.58441,NaN,88.50397,60.85935,66.90208,84.84532,NaN
L_IB33___BioAssayId=72902Name=050705MJA_U133_2.0_IB03.CEL,13.79470,NaN,1358.15430,45.66821,8.50909,NaN,17.39715,18.87131,40.65568,99.47049,...,NaN,NaN,57.62479,840.18787,NaN,90.49266,51.56433,62.14825,80.57075,NaN
L_IB36___BioAssayId=72905Name=050705MJA_U133_2.0_IB06.CEL,13.70568,NaN,1152.48254,46.30679,9.29537,NaN,22.76678,14.42896,0.36647,69.15773,...,NaN,17.52186,43.80604,389.94690,NaN,53.26638,13.28645,42.16255,59.41403,NaN


In [6]:
# Save
mapped_compendium.to_csv(mapped_compendium_filename, sep="\t")

In [7]:
# Normalize data
process.normalize_compendium(
    mapped_compendium_filename, normalized_compendium_filename, scaler_filename
)

### Select and process template data

1. Get gene expression associated with `project_id`, which was manually selected by the user and specified in the config file.

Note: The data is not normalized so that we can perform DE analysis in next notebook

In [8]:
# Note: This is the only notebook using this function, so for now it is included here
# Get sample ids associated with selected project id
def get_sample_ids(experiment_id, mapping_filename):
    """
    Return sample ids for a given experiment id

    """
    # Read in metadata
    metadata = pd.read_csv(mapping_filename, header=0)
    metadata.set_index("Experiment id", inplace=True)

    selected_metadata = metadata.loc[experiment_id]
    sample_ids = list(selected_metadata[metadata_colname])

    return sample_ids


# metadata file with mapping from experiment to sample
experiment_to_sample_metadata_filename = os.path.join(
    base_dir, dataset_name, "data", "metadata", "experiment_sample_annotations.csv"
)

sample_ids = get_sample_ids(project_id, experiment_to_sample_metadata_filename)

# Get expression data
template_mapped = mapped_compendium.loc[sample_ids]
print(template_mapped.shape)

# Save
template_mapped.to_csv(mapped_template_filename, sep="\t")

(36, 9240)


### Train VAE

In [9]:
# Create VAE directories if needed
output_dirs = [
    os.path.join(base_dir, dataset_name, "models"),
    os.path.join(base_dir, dataset_name, "logs"),
]

NN_architecture = params["NN_architecture"]

# Check if NN architecture directory exist otherwise create
for each_dir in output_dirs:
    sub_dir = os.path.join(each_dir, NN_architecture)
    os.makedirs(sub_dir, exist_ok=True)

In [10]:
# Train VAE on new compendium data
train_vae_modules.train_vae(config_filename, normalized_compendium_filename)

input dataset contains 32476 samples and 9240 genes
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
tracking <tf.Variable 'Variable:0' shape=() dtype=float32> beta
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/keras/engine/training_utils.py:819: UserWarning: Output custom_variational_layer_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to custom_variational_layer_1.
  'be expecting any data to be passed to {0}.'.format(name))



Train on 24357 samples, validate on 8119 samples
Epoch 1/40
24357/24357 [==============================] - 1935s 79ms/step - loss: nan - val_loss: nan
Epoch 2/40
24357/24357 [==============================] - 1941s 80ms/step - loss: nan - val_loss: nan
Epoch 3/40
  560/24357 [..............................] - ETA: 31:52 - loss: nan

KeyboardInterrupt: 